In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
import tensorflow
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy

In [3]:
real_df = pd.read_csv("cosas_finales/casos_semireal_new_all.txt", error_bad_lines=False)
real_df

C:\Users\migue\AppData\Local\Temp\ipykernel_14108\965987508.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/casos_semireal_new_all.txt", error_bad_lines=False)


,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipoProblema
0,2584,0.0107,11629,0.0009,24551,0.0008,0
1,1055,0.0031,4750,0.0015,10028,0.0012,0
2,2989,0.0015,13450,0.0011,28395,0.0011,0
3,197,0.0025,886,0.0011,1871,0.0010,0
4,754,0.0013,3393,0.0010,7164,0.0009,0
...,...,...,...,...,...,...,...
11993,97,556.7960,438,50874.6000,926,483428.0000,5
11994,99,595.7910,447,55007.4000,945,401368.0000,5
11995,97,387.8500,438,34860.9000,926,311789.0000,5
11996,98,365.8730,442,34268.2000,933,309820.0000,5


In [4]:
#real_df = real_df.drop(['N', 'N.1', 'N.2'], axis=1)
x_train = real_df.iloc[:, 1:7]
#x_train = real_df.iloc[:, 0:3]

x_train

,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipoProblema
0,0.0107,11629,0.0009,24551,0.0008,0
1,0.0031,4750,0.0015,10028,0.0012,0
2,0.0015,13450,0.0011,28395,0.0011,0
3,0.0025,886,0.0011,1871,0.0010,0
4,0.0013,3393,0.0010,7164,0.0009,0
...,...,...,...,...,...,...
11993,556.7960,438,50874.6000,926,483428.0000,5
11994,595.7910,447,55007.4000,945,401368.0000,5
11995,387.8500,438,34860.9000,926,311789.0000,5
11996,365.8730,442,34268.2000,933,309820.0000,5


In [6]:
y_train = []
for i in range(0,len(real_df)):
  high = real_df.tipoProblema.values[i]
  if high == 0:
    y_train.append([1,0,0,0,0,0])
  elif high == 1:
    y_train.append([0,1,0,0,0,0])
  elif high == 2:
    y_train.append([0,0,1,0,0,0])
  elif high == 3:
    y_train.append([0,0,0,1,0,0])
  elif high == 4:
    y_train.append([0,0,0,0,1,0])
  elif high == 5:
    y_train.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipoProblema[i]))

#len(y_train)

AttributeError: 'DataFrame' object has no attribute 'tipo'

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
#y_train

In [ ]:
model = load_model("cosas_finales/model3-8(2).h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 200)            161600    
                                                                 
 dropout (Dropout)           (None, 6, 200)            0         
                                                                 
 lstm_1 (LSTM)               (None, 6, 200)            320800    
                                                                 
 dropout_1 (Dropout)         (None, 6, 200)            0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 200)            320800    
                                                                 
 dropout_2 (Dropout)         (None, 6, 200)            0         
                                                                 
 lstm_3 (LSTM)               (None, 200)               3

In [ ]:
real_df_preds = model.predict(x_train)
#real_df_preds

In [ ]:
err = real_df_preds - y_train
#err

In [ ]:
real_preds = []
for i in range(0,len(real_df_preds)):
  p = real_df_preds[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])

#real_preds

In [ ]:
err = real_preds - y_train
err

array([[-1.,  1.,  0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  1.],
       [ 1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.],
       [ 0.,  0.,  0.,  0., -1.,  1.]])

añadir O(1).
t2 - t1 y t3 - t1 => plot (esto no hace falta era para el debug)

In [ ]:
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    print(str(real_df.name.values[i]) + " sol: " + numToType(real_df.tipo.values[i]) + "  pred: " + numToType(np.argmax(real_preds[i])))
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

-114 sol: CTE  pred: LOG
-cajasBolas sol: LOG  pred: CTE
-167fractales sol: LOG  pred: CTE
--OJ10019 sol: LOG  pred: N^3
--323 sol: LOG  pred: CTE
niniosPrimero sol: N  pred: N^3
--cazaTesoros sol: N^2  pred: N^3
tiroPalindromo sol: N^2  pred: N^3
---probTestUsefulErr sol: N^2  pred: N^3
9 / 15 => real error% of 60.0


In [ ]:
expected_train = []
for i in range(0,len(real_df)):
  high = real_df.tipoProblema.values[i]
  if high == 0:
    expected_train.append([1,0,0,0,0,0])
  elif high == 1:
    expected_train.append([0,1,0,0,0,0])
  elif high == 2:
    expected_train.append([0,0,1,0,0,0])
  elif high == 3:
    expected_train.append([0,0,0,1,0,0])
  elif high == 4:
    expected_train.append([0,0,0,0,1,0])
  elif high == 5:
    expected_train.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))


In [ ]:
#error teniendo en cuenta la aplicacion real del programa ( no todos los fallos son fallos )
#ejemplo: espero un lineal y entregan un n2, lo interpreto como n3, el resultado seria "no es lineal" y no seria un fallo
#creo que este tipo de error_count solo se puede calcular aplicando el modelo a un problema donde espero un resultado y entregan otro, no se como hacerlo aqui

#y_train = []            #tipo de ejecucion real del programa
#expected_train = []     #tipo de ejecucion esperado por el problema
predictions_train = real_preds  #ordenes predecidos por la ia

correct = 0
partial_corrects = 0
errors = 0
for i in range(0, len(real_preds)):
  guess_id = np.argmax(predictions_train[i])
  answer_id = np.argmax(y_train[i])
  expected_id = np.argmax(expected_train[i])
  if guess_id == answer_id:
    correct += 1
  elif guess_id > answer_id and expected_id != answer_id:
    partial_corrects += 1
  else:
    errors += 1

print(str(errors) + " / " + str(len(real_preds)) + " => useful error% of " + str(errors/len(real_preds)*100))
print(str(correct) + " + " + str(partial_corrects) + " = " + str(correct+partial_corrects)  + " / " + str(len(real_preds)) + " => useful guesses% of " + str((correct+partial_corrects)/len(real_preds)*100))


8 / 15 => useful error% of 53.333333333333336
6 + 1 = 7 / 15 => useful guesses% of 46.666666666666664
